## **PROBLEMA DE OPTIMIZACIÓN DE LA DIETA MILITAR**

### **Descripción:**

El objetivo de este problema es determinar la cantidad óptima de raciones de alimento que deben ser consumidas por un soldado para cumplir con sus requerimientos nutricionales diarios a un costo mínimo. Cada ración de alimento tiene un costo asociado y valores específicos de nutrientes esenciales. Las restricciones están determinadas por los requisitos nutricionales diarios mínimos y máximos establecidos para garantizar el adecuado rendimiento físico y mental del personal militar. Las raciones pueden ser fracciones, un soldado puede comer 1.62 raciones de cualquier alimento.

### **Datos:**

-   **Raciones de Alimento**: Una lista de raciones de alimento disponibles para el consumo, cada una con su respectivo costo por unidad y las cantidades mínimas y máximas que se pueden consumir por día.

-   **Nutrientes**: Una lista de nutrientes esenciales que deben ser consumidos diariamente, cada uno con su cantidad mínima y máxima recomendada.

-   **Valores Nutricionales de las Raciones de Alimento**: Una matriz que indica la cantidad de cada nutriente presente en cada ración de alimento.

### **Objetivo:**

Minimizar el costo total de las raciones de alimento consumidas mientras se cumple con las cantidades recomendadas de cada nutriente.

### **Restricciones:**

1. Las cantidades de raciones de alimento consumidas deben estar entre las cantidades mínimas y máximas definidas para cada ración de alimento.
2. La cantidad total de cada nutriente consumido a partir de todas las raciones de alimento debe estar entre las cantidades mínimas y máximas definidas para cada nutriente.

### **Raciones de Alimento**

|       Alimento       | Costo_Unitario | qmin | qmax |
| :------------------: | :------------: | :--: | :--: |
|     Pollo asado      |      0.84      |  1   |  14  |
|   Pasta con salsa    |      0.78      |  0   |  14  |
|    Tomate fresco     |      0.27      |  1   |  10  |
|    Manzana fresca    |      0.24      |  1   |  9   |
|         Uvas         |      0.32      |  2   |  13  |
| Galletas energéticas |      0.03      |  2   |  15  |
|   Leche descremada   |      0.23      |  2   |  12  |
|  Cereal con frutas   |      0.34      |  0   |  9   |
|      Salchichas      |      0.31      |  1   |  15  |

### **Nutrientes**

|    Nutiente     | qmin | qmax  |
| :-------------: | :--: | :---- |
|    Calorías     | 2000 | 2500  |
|     Calcio      | 800  | 1600  |
|     Hierro      |  10  | 30    |
|   Vitamina A    | 5000 | 50000 |
| Fibra dietética |  25  | 100   |
|  Carbohidratos  |  0   | 300   |
|    Proteína     |  50  | 100   |

### **Valores Nutricionales de las Raciones de Alimento**

|         A/N          | Calorías | Calcio | Hierro | Vit_A  | Fibra dietética | Carbohidratos | Proteína |
| :------------------: | :------: | :----: | :----: | :----: | :-------------: | :-----------: | :------: |
|     Pollo asado      |  277.4   |  21.9  |  1.8   |  77.4  |        0        |       0       |   42.2   |
|   Pasta con salsa    |  358.2   |  80.2  |  2.3   | 3055.2 |      11.6       |     58.3      |   8.2    |
|    Tomate fresco     |   25.8   |  6.2   |  0.6   | 766.3  |       1.4       |      5.7      |    1     |
|    Manzana fresca    |   81.4   |  9.7   |  0.2   |  73.1  |       3.7       |      21       |   0.3    |
|         Uvas         |   15.1   |  3.4   |  0.1   |   24   |       0.2       |      4.1      |   0.2    |
| Galletas energéticas |   78.1   |  6.2   |  0.4   | 101.8  |        0        |      9.3      |   0.9    |
|   Leche descremada   |  121.2   | 296.7  |  0.1   | 500.2  |        0        |     11.7      |   8.1    |
|  Cereal con frutas   |  115.1   |  12.9  |  16.8  | 1250.2 |        4        |     27.9      |    4     |
|      Salchichas      |  242.1   |  23.5  |  2.3   |   0    |        0        |      18       |   10.4   |

### **¿Cómo cambiaría el problema si? (Resolver en notebbok distinto)**

1. Queremos que la ingesta de salchichas represente mínimo el 12% de la ingesta total de alimentos
2. Queremos maximizar la ingesta de proteína como primera prioridad y los costos como segunda


In [1]:
import pandas
from docplex.mp.model import Model
import plotly.graph_objects as go
import gurobipy as gp
from gurobipy import GRB

In [2]:
df_foods = pandas.read_excel("./data.xlsx", sheet_name="Raciones de Alimento", index_col=0)
df_nutrients = pandas.read_excel("./data.xlsx", sheet_name="Nutrientes", index_col=0)
df_food_nutrient = pandas.read_excel("./data.xlsx", sheet_name="Valores Nutricionales", index_col=0)

In [3]:
df_foods

costo unitario  qmin  qmax
alimento                                        
Pollo asado                     0.84     1    14
Pasta con salsa                 0.78     0    14
Tomate fresco                   0.27     1    10
Manzana fresca                  0.24     1     9
Uvas                            0.32     2    13
Galletas energéticas            0.03     2    15
Leche descremada                0.23     2    12
Cereal con frutas               0.34     0     9
Salchichas                      0.31     1    15

In [4]:
df_foods.index.to_list()

['Pollo asado',
 'Pasta con salsa',
 'Tomate fresco',
 'Manzana fresca',
 'Uvas',
 'Galletas energéticas',
 'Leche descremada',
 'Cereal con frutas',
 'Salchichas']

In [5]:
df_nutrients

qmin   qmax
nutriente                 
Calorías       2000   2500
Calcio          800   1600
Hierro           10     30
Vitamina A     5000  50000
Fibra            25    100
Carbohidratos     0    300
Proteína         50    100

**Conjuntos**


$$ \text{FOODS} : \text{Conjunto de alimentos} $$
$$ \text{NUTRIENTS} : \text{Conjunto de nutrientes} $$


In [6]:
FOODS = df_foods.index.to_list()
NUTRIENTS = df_nutrients.index.to_list()

**Parámetros**


$$ \text{c}_f \in \mathbb{R}^{+}: \text{Costo por unidad del alimento } f \in \text{F} $$
$$ \text{food\_min}_f \in \mathbb{R}^{+}: \text{Cantidad mínima permitida del alimento } f \in \text{F} $$
$$ \text{food\_max}_f \in \mathbb{R}^{+}: \text{Cantidad máxima permitida del alimento } f \in \text{F} $$
$$ \text{nutrient\_min}_n \in \mathbb{R}^{+}: \text{Cantidad mínima requerida del nutriente } n \in \text{N} $$
$$ \text{nutrient\_max}_n \in \mathbb{R}^{+}: \text{Cantidad máxima permitida del nutriente } n \in \text{N} $$
$$ \text{a}_{fn} \in \mathbb{R}^{+}: \text{Cantidad del nutriente } n \in \text{N} \text{ en el alimento } f \in \text{F} $$


In [7]:
## Usaremos los dataframe
print(df_foods.loc["Pollo asado", "costo unitario"])
print(df_foods.loc["Pollo asado", "qmin"])
print(df_foods.loc["Pollo asado", "qmax"])
print(df_nutrients.loc["Calcio", "qmin"])
print(df_nutrients.loc["Calcio", "qmax"])

0.84
1
14
800
1600


**Variables de decisión**

$$ x_f \in \mathbb{R}^{+}: \text{Cantidad del alimento } f \in \text{FOODS} \text{ a consumir} $$
$$ y_n \in \mathbb{R}^{+}: \text{Cantidad del nutriente } n \in \text{NUTRIENTS} \text{ a consumir} $$


In [8]:
# Creación del modelo
model = gp.Model(name="Militar Diet")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-28


In [9]:
# Variables
x = model.addVars(FOODS, vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(NUTRIENTS, vtype=GRB.CONTINUOUS, name="y")

**Función Objetivo**


$$ \text{minimizar FO} = \sum_{f \in \text{FOODS}} c_f \cdot x_f $$


In [10]:
# Función objetivo: Minimizar el costo total de los alimentos
model.setObjective(gp.quicksum(x[f] * df_foods.loc[f, "costo unitario"] for f in FOODS), GRB.MINIMIZE)

**Restricciones**

1. Respetar la cantidad mínima y máxima de cada alimento:
$$ \forall f \in \text{FOODS:} \quad x_f \geq \text{food\_min}_f $$
$$ \forall f \in \text{FOODS:} \quad x_f \leq \text{food\_max}_f $$

2. Satisfacer las necesidades de nutrientes:
$$ \forall n \in \text{NUTRIENTS:} \quad y_n = \sum_{f \in \text{FOODS}} a_{fn} \cdot x_f $$
$$ \forall n \in \text{NUTRIENTS:} \quad y_n \geq \text{nutrient\_min}_n  $$
$$ \forall n \in \text{NUTRIENTS:} \quad y_n \leq \text{nutrient\_max}_n  $$


In [11]:
# 1. Respetar la cantidad mínima y máxima de cada alimento
for f in FOODS:
    model.addConstr(x[f] >= df_foods.loc[f, "qmin"], f"min_{f}")
    
# model.addConstrs((x[f] >= df_foods.loc[f, "qmin"] for f in FOODS))

for f in FOODS:
    model.addConstr(x[f] <= df_foods.loc[f, "qmax"], f"max_{f}")

for n in NUTRIENTS:
    model.addConstr(y[n] == gp.quicksum(df_food_nutrient.loc[f, n] * x[f] for f in FOODS), f"nut_{n}")

for n in NUTRIENTS:
    model.addConstr(y[n] >= df_nutrients.loc[n, "qmin"], f"nut_min_{n}")

for n in NUTRIENTS:
    model.addConstr(y[n] <= df_nutrients.loc[n, "qmax"], f"nut_max_{n}")

In [12]:
# Resolviendo el modelo
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 39 rows, 16 columns and 96 nonzeros
Model fingerprint: 0xf6ae9c1c
Coefficient statistics:
  Matrix range     [1e-01, 3e+03]
  Objective range  [3e-02, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 32 rows and 2 columns
Presolve time: 0.01s
Presolved: 7 rows, 14 columns, 62 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8200000e+00   8.455938e+01   0.000000e+00      0s
       3    4.2627127e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.262712703e+00


In [13]:
model.ObjVal

4.2627127025475735

In [14]:
# Imprimiendo la solución
print("FOODS")
for f in FOODS:
    print(f"{f}: {x[f].X}")

print("\nNUTRIENTS")
for n in NUTRIENTS:
    print(f"{n}: {y[n].X}")

FOODS
Pollo asado: 1.0
Pasta con salsa: 1.681034482758621
Tomate fresco: 1.0
Manzana fresca: 1.0
Uvas: 2.0
Galletas energéticas: 6.383526866528322
Leche descremada: 2.0
Cereal con frutas: 0.0
Salchichas: 1.0

NUTRIENTS
Calorías: 2000.0
Calcio: 835.896832089717
Hierro: 11.719790056956157
Vitamina A: 7750.939586736722
Fibra: 25.0
Carbohidratos: 233.671110203541
Proteína: 90.02965693849619


In [17]:
FOODS

['Pollo asado',
 'Pasta con salsa',
 'Tomate fresco',
 'Manzana fresca',
 'Uvas',
 'Galletas energéticas',
 'Leche descremada',
 'Cereal con frutas',
 'Salchichas']

In [18]:
[x[f].X for f in FOODS]

[1.0, 1.681034482758621, 1.0, 1.0, 2.0, 6.383526866528322, 2.0, 0.0, 1.0]

In [22]:
fig = go.Figure()

fig.add_trace(go.Bar(x=FOODS, y=[x[f].X for f in FOODS], text=[f"{x[f].X:.2f}" for f in FOODS],))

for idx, f in enumerate(FOODS):
    fig.add_shape(
        type="line",
        x0=idx - 0.4,
        x1=idx + 0.4,
        y0=df_foods.loc[f, "qmin"],
        y1=df_foods.loc[f, "qmin"],
        line=dict(color="blue", width=2, dash="dash"),
    )
    
    fig.add_shape(
        type="line",
        x0=idx - 0.4,
        x1=idx + 0.4,
        y0=df_foods.loc[f, "qmax"],
        y1=df_foods.loc[f, "qmax"],
        line=dict(color="blue", width=2, dash="dash"),
    )
    
fig.update_layout(
    title="CONSUMO DE RACIONES DE ALIMENTOS",
    xaxis_title="Alimentos",
    yaxis_title="Raciones",
    xaxis={"type": "category"},
    showlegend=False,
    width=800,
    template="ggplot2",
)


fig.show()

In [16]:
fig = go.Figure()

nutrients_percentages = [(y[n].X) / (df_nutrients.loc[n, "qmax"]) for n in NUTRIENTS]
fig.add_trace(
    go.Bar(
        x=NUTRIENTS,
        y=nutrients_percentages,
        text=[f"{p:.2%}" for p in nutrients_percentages],
        textposition="outside",
        marker_color="goldenrod"
    )
)

for idx, n in enumerate(NUTRIENTS):
    fig.add_shape(
        type="line",
        x0=idx - 0.4,
        x1=idx + 0.4,
        y0=df_nutrients.loc[n, "qmin"] / (df_nutrients.loc[n, "qmax"]),
        y1=df_nutrients.loc[n, "qmin"] / (df_nutrients.loc[n, "qmax"]),
        line=dict(color="blue", width=2, dash="dash"),
    )


# Configuraciones adicionales de la gráfica
fig.update_layout(
    title="POECENTAJES DE CUMPLIMIENTO NUTRIENTES",
    xaxis_title="Nutrientes",
    yaxis_title="Porcentaje ",
    xaxis={"type": "category"},
    showlegend=False,
    width=800,
    height=500,
    template="ggplot2",
)

# Mostrar la gráfica
fig.show()

| Concepto                        | Descripción                                                 | Ecuación                                                                                      | Código Python (Gurobipy)                                                                                              |
| :-----------------------------: | ----------------------------------------------------------- | --------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------- |
|  **SET**                        | Conjunto de alimentos                                       | $$ \text{FOODS} $$                                                                            | `FOODS = [...]`                                                                                                 |
|  **SET**                        | Conjunto de nutrientes                                      | $$ \text{NUTRIENTS} $$                                                                        | `NUTRIENTS = [...]`                                                                                             |
| **PARAM**                       | Costo por unidad del alimento \(f\)                         | $$ \text{c}_f \in \mathbb{R}^{+} $$                                                          | `df_foods.loc[f, 'costo']`                                                                                             |
| **PARAM**                       | Cantidad mínima permitida del alimento \(f\)                | $$ \text{food\_min}_f \in \mathbb{R}^{+} $$                                                   | `df_foods.loc[f, 'qmin']`                                                                                              |
| **PARAM**                       | Cantidad máxima permitida del alimento \(f\)                | $$ \text{food\_max}_f \in \mathbb{R}^{+} $$                                                   | `df_foods.loc[f, 'qmax']`                                                                                              |
| **PARAM**                       | Cantidad mínima requerida del nutriente \(n\)               | $$ \text{nutrient\_min}_n \in \mathbb{R}^{+} $$                                               | `df_nutrients.loc[n, 'qmin']`                                                                                          |
| **PARAM**                       | Cantidad máxima permitida del nutriente \(n\)               | $$ \text{nutrient\_max}_n \in \mathbb{R}^{+} $$                                               | `df_nutrients.loc[n, 'qmax']`                                                                                          |
| **PARAM**                       | Cantidad del nutriente \(n\) en el alimento \(f\)           | $$ \text{a}_{fn} \in \mathbb{R}^{+} $$                                                       | `df_nutrient_content`                                                                                           |
|  **VAR**                        | Variables de alimentos                                      | $$ x_f \in \mathbb{R}^{+} $$                                                                  | `x = model.addVars(FOODS, lb=0, name="x", vtype=GRB.CONTINUOUS)`                                                                      |
|  **VAR**                        | Variables de nutrientes                                     | $$ y_n \in \mathbb{R}^{+} $$                                                                  | `y = model.addVars(NUTRIENTS, lb=0, name="y", vtype=GRB.CONTINUOUS)`                                                                  |
|  **FO**                         | Función objetivo: Minimizar el costo total de los alimentos | $$ \text{minimizar FO} = \sum_{f \in \text{FOODS}} c_f \cdot x_f $$                           | `model.setObjective(gp.quicksum(x[f] * df_foods.loc[f, 'costo'] for f in FOODS), GRB.MINIMIZE)`                 |
| **CONST**                       | Restricción mínima de alimentos                             | $$ \forall f \in \text{FOODS:} \quad x_f \geq \text{food\_min}_f $$                           | `model.addConstrs((x[f] >= df_foods.loc[f, 'qmin'] for f in FOODS), name="min_food")`                           |
| **CONST**                       | Restricción máxima de alimentos                             | $$ \forall f \in \text{FOODS:} \quad x_f \leq \text{food\_max}_f $$                           | `model.addConstrs((x[f] <= df_foods.loc[f, 'qmax'] for f in FOODS), name="max_food")`                           |
| **CONST**                       | Relación nutrientes-alimentos                               | $$ \forall n \in \text{NUTRIENTS:} \quad y_n = \sum_{f \in \text{FOODS}} a_{fn} \cdot x_f $$  | `model.addConstrs((y[n] == gp.quicksum(df_nutrient_content.loc[f, n] * x[f] for f in FOODS) for n in NUTRIENTS), name="nutrient_balance")` |
| **CONST**                       | Restricción mínima de nutrientes                            | $$ \forall n \in \text{NUTRIENTS:} \quad y_n \geq \text{nutrient\_min}_n $$                   | `model.addConstrs((y[n] >= df_nutrients.loc[n, 'qmin'] for n in NUTRIENTS), name="min_nutrient")`               |
| **CONST**                       | Restricción máxima de nutrientes                            | $$ \forall n \in \text{NUTRIENTS:} \quad y_n \leq \text{nutrient\_max}_n $$                    | `model.addConstrs((y[n] <= df_nutrients.loc[n, 'qmax'] for n in NUTRIENTS), name="max_nutrient")`               |
